In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def encontrar_data_anterior_mais_proxima(lista_datas_str, data_referencia):
  datas = []
  for data_str in lista_datas_str:
    try:
      datas.append(datetime.strptime(data_str,  "%Y-%m-%d %H:%M:%S.%f"))
    except ValueError:
      print(f"Data inválida: {data_str}")
  datas_anteriores = [data for data in datas if data <= data_referencia]

  if datas_anteriores:
    return max(datas_anteriores).strftime( "%Y-%m-%d %H:%M:%S.%f")[:-3]
  else:
    return None

def encontrar_data_posterior_mais_proxima(lista_datas_str, data_referencia):
  datas = []
  for data_str in lista_datas_str:
    try:
      datas.append(datetime.strptime(data_str, "%Y-%m-%d %H:%M:%S.%f"))
    except ValueError:
      print(f"Data inválida: {data_str}")
  datas_posteriores = [data for data in datas if data >= data_referencia]

  if datas_posteriores:
    return min(datas_posteriores).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
  else:
    return None

dia = mes = 0
listaQtdAlocacoes = []
colunas = ['RN','CLIENTE', 'CD_ALOCACAO', 'DATA_HORA_VAZAO', 'TURNO', 'DATA_SAIDA_CAMINHAO','DATA_CHEGADA_CAMINHAO',
           'DESCRICAO_TIPO_CARGA','PESO_PADRAO','DISTANCIA','VAZAO_FRENTE','CLASSIFICACAO']

datasetAjustado = pd.DataFrame(columns=colunas)
for i in range(3,12,1):
  for j in range(1,32,1):
    if j == 31 and i == 11:
      break
    if (i == 4 or i == 6 or i == 9 or i == 11) and (j == 31):
      continue
    dia = str(j)
    if j < 10:
      dia = "0" + str(j)
      mes = str(i)
    if i < 10:
      mes = "0" + str(i)
    url = f"https://raw.githubusercontent.com/RafaelGirardin/ModeloPredicaoFazenda/refs/heads/master/CERRADINHO/{i}/{mes}-{dia}.csv"
    datasetOriginal= pd.read_csv(url, names=colunas, skiprows=1, delimiter=',')
    datasetOriginal['CLASSIFICACAO'] = "Erro"
    datasetOriginal['TEMPO_INFERIOR'] = datasetOriginal['MARGEM_INFERIOR'] = datasetOriginal['MARGEM_SUPERIOR'] = 0
    listaQtdAlocacoes.append(len(datasetOriginal['CD_ALOCACAO']))
    if len(datasetOriginal['CD_ALOCACAO']) == 0:
      continue
    percVazio = (len(datasetOriginal.query('VAZAO_FRENTE.isnull()', engine='python')) + len(datasetOriginal.query('VAZAO_FRENTE == 0', engine='python')))/len(datasetOriginal['VAZAO_FRENTE'])
    if percVazio >= 0.20:
      continue
    print(dia + "-" + mes)
    D1 = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO']).quantile(0.03)
    D9 = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO']).quantile(0.97)
    datasetOriginal['DATA_CHEGADA_CAMINHAO'] = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO'])
    datasetOriginal = datasetOriginal[datasetOriginal['DATA_CHEGADA_CAMINHAO'] >= D1]
    datasetOriginal = datasetOriginal[datasetOriginal['DATA_CHEGADA_CAMINHAO'] <= D9]
    datasetOriginal['DATA_CHEGADA_CAMINHAO'] = datasetOriginal['DATA_CHEGADA_CAMINHAO'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
    datasetOriginal = datasetOriginal.reset_index(drop=True)

    datasetDoDiaComVazoesPreenchidas = datasetOriginal.query('(not VAZAO_FRENTE.isnull()) and VAZAO_FRENTE > 0', engine='python')
    for vazao in range(0, len(datasetOriginal['VAZAO_FRENTE'])):
      if np.isnan(datasetOriginal['VAZAO_FRENTE'][vazao]) or datasetOriginal['VAZAO_FRENTE'][vazao] == 0:
        distancia = datasetOriginal['DISTANCIA'][vazao]
        datasetDoDiaComVazoesPreenchidasParaUmaFazenda = datasetDoDiaComVazoesPreenchidas.loc[(datasetOriginal['DISTANCIA'] == distancia)]
        datetimeDataVazaoNula = datetime.strptime(datasetOriginal['DATA_HORA_VAZAO'][vazao],"%Y-%m-%d %H:%M:%S.%f")
        stringDataVazaoNaoNulaAnteriorMaisProxima = encontrar_data_anterior_mais_proxima(datasetDoDiaComVazoesPreenchidasParaUmaFazenda['DATA_HORA_VAZAO'], datetimeDataVazaoNula)
        stringDataVazaoNaoNulaPosteriorMaisProxima = encontrar_data_posterior_mais_proxima(datasetDoDiaComVazoesPreenchidasParaUmaFazenda['DATA_HORA_VAZAO'], datetimeDataVazaoNula)
        if stringDataVazaoNaoNulaAnteriorMaisProxima is None and stringDataVazaoNaoNulaPosteriorMaisProxima is None:
          if len(datasetDoDiaComVazoesPreenchidas) == 0:
            break
          somaVazoes = 0
          for vazaoNaoNula in range(0, len(datasetDoDiaComVazoesPreenchidas)):
            somaVazoes += datasetDoDiaComVazoesPreenchidas.iloc[vazaoNaoNula]['VAZAO_FRENTE']
          mediaVazoesDiferentesFazendas = somaVazoes/len(datasetDoDiaComVazoesPreenchidas)
          datasetOriginal['VAZAO_FRENTE'][vazao] = mediaVazoesDiferentesFazendas
          continue
        elif stringDataVazaoNaoNulaAnteriorMaisProxima is None:
          stringDataVazaoNaoNulaAnteriorMaisProxima = stringDataVazaoNaoNulaPosteriorMaisProxima
        elif stringDataVazaoNaoNulaPosteriorMaisProxima is None:
          stringDataVazaoNaoNulaPosteriorMaisProxima = stringDataVazaoNaoNulaAnteriorMaisProxima
        linhasComVazoesNaoNulasAnterioresMaisProximas = datasetOriginal.loc[datasetOriginal['DATA_HORA_VAZAO'] == stringDataVazaoNaoNulaAnteriorMaisProxima]
        linhasComVazoesNaoNulasPosterioresMaisProximas = datasetOriginal.loc[datasetOriginal['DATA_HORA_VAZAO'] == stringDataVazaoNaoNulaPosteriorMaisProxima]
        vazaoNaoNulaAnteriorMaisProxima = list(linhasComVazoesNaoNulasAnterioresMaisProximas['VAZAO_FRENTE'])[0]
        vazaoNaoNulaPosteriorMaisProxima = list(linhasComVazoesNaoNulasPosterioresMaisProximas['VAZAO_FRENTE'])[0]
        mediaVazoesNaoNulasMaisProximas = (vazaoNaoNulaAnteriorMaisProxima + vazaoNaoNulaPosteriorMaisProxima)/2
        datasetOriginal['VAZAO_FRENTE'][vazao] = mediaVazoesNaoNulasMaisProximas
    for alocacao in range(1, len(datasetOriginal['CD_ALOCACAO'])):
      if pd.isna(datasetOriginal['DATA_CHEGADA_CAMINHAO'][alocacao]):
        datasChegadasPreenchidas = datasetOriginal.loc[datasetOriginal['DATA_CHEGADA_CAMINHAO'].notna()]
        datasChegadasPreenchidasParaUmaFazenda = datasChegadasPreenchidas.loc[datasChegadasPreenchidas['DISTANCIA'] == datasetOriginal['DISTANCIA'][alocacao]]
        somaTempoDasViagens = mediaTempoDasViagens = 0
        if len(datasChegadasPreenchidasParaUmaFazenda) != 0:
          for data in range(0, len(datasChegadasPreenchidasParaUmaFazenda)):
            somaTempoDasViagens += (datetime.strptime(datasChegadasPreenchidasParaUmaFazenda.iloc[data]['DATA_CHEGADA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f") - datetime.strptime(datasChegadasPreenchidasParaUmaFazenda.iloc[data]['DATA_SAIDA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f")).total_seconds()/60
            mediaTempoDasViagens = somaTempoDasViagens/len(datasChegadasPreenchidasParaUmaFazenda)
        else:
          for data in range(0, len(datasChegadasPreenchidas)):
            somaTempoDasViagens += (datetime.strptime(datasChegadasPreenchidas.iloc[data]['DATA_CHEGADA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f") - datetime.strptime(datasChegadasPreenchidas.iloc[data]['DATA_SAIDA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f")).total_seconds()/60
            mediaTempoDasViagens = somaTempoDasViagens/len(datasChegadasPreenchidas)
        tempoPrevistoViagem = timedelta(minutes=mediaTempoDasViagens)
        datasetOriginal['DATA_CHEGADA_CAMINHAO'][alocacao] = (datetime.strptime(datasetOriginal['DATA_SAIDA_CAMINHAO'][alocacao],"%Y-%m-%d %H:%M:%S.%f") + tempoPrevistoViagem).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    datasetOriginal['DATA_CHEGADA_CAMINHAO'] = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO'])
    datasetOriginal= datasetOriginal.sort_values(by='DATA_CHEGADA_CAMINHAO', ascending=True)
    datasetOriginal['DATA_CHEGADA_CAMINHAO'] = datasetOriginal['DATA_CHEGADA_CAMINHAO'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    amplitudeDeHorariosDaAlocacao = max([datetime.strptime(data, "%Y-%m-%d %H:%M:%S.%f") for data in datasetOriginal['DATA_SAIDA_CAMINHAO']]) - min([datetime.strptime(data, "%Y-%m-%d %H:%M:%S.%f") for data in datasetOriginal['DATA_SAIDA_CAMINHAO']])
    amplitudeEmMinutosDeHorariosDaAlocacao = float((amplitudeDeHorariosDaAlocacao.total_seconds()) / 60)
    margemSuperior = amplitudeEmMinutosDeHorariosDaAlocacao/(len(datasetOriginal['DATA_SAIDA_CAMINHAO']) - 1) + (amplitudeEmMinutosDeHorariosDaAlocacao)/(len(datasetOriginal['DATA_SAIDA_CAMINHAO']) - 1)*0.4
    margemInferior =  amplitudeEmMinutosDeHorariosDaAlocacao/(len(datasetOriginal['DATA_SAIDA_CAMINHAO']) - 1) - (amplitudeEmMinutosDeHorariosDaAlocacao)/(len(datasetOriginal['DATA_SAIDA_CAMINHAO']) - 1)*0.4
    for m in range(1,len(datasetOriginal['DATA_CHEGADA_CAMINHAO'])):
      tempoInferior = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO'][m], format='%Y-%m-%d %H:%M:%S.%f') - pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO'][m-1], format='%Y-%m-%d %H:%M:%S.%f')
      if (tempoInferior).seconds/60 >=  margemInferior and (tempoInferior).seconds/60 <= margemSuperior:
        datasetOriginal['CLASSIFICACAO'][m] = "A"
      elif (tempoInferior).seconds/60 >=  margemInferior/1.25 and (tempoInferior).seconds/60 <= margemSuperior*1.25:
        datasetOriginal['CLASSIFICACAO'][m] = "B"
      elif (tempoInferior).seconds/60 >=  margemInferior/2.0 and (tempoInferior).seconds/60 <= margemSuperior*2.0:
        datasetOriginal['CLASSIFICACAO'][m] = "C"
      elif (tempoInferior).seconds/60 >=  margemInferior/4.0 and (tempoInferior).seconds/60 <= margemSuperior*4.0:
        datasetOriginal['CLASSIFICACAO'][m] = "D"
      else:
        datasetOriginal['CLASSIFICACAO'][m] = "E"
      datasetOriginal['TEMPO_INFERIOR'][m] = tempoInferior.seconds/60
      datasetOriginal['MARGEM_INFERIOR'][m] = margemInferior
      datasetOriginal['MARGEM_SUPERIOR'][m] = margemSuperior
    datasetAjustado = pd.concat([datasetAjustado, datasetOriginal],ignore_index=True)
datasetAjustado = datasetAjustado[datasetAjustado['CLASSIFICACAO'] != 'Erro']
print(datasetAjustado.shape)
datasetAjustado.head(12000)

25-03
16-04
17-04
26-04
28-04
15-05
22-05
23-05
26-05
11-06
27-06
07-07
29-07
30-07
01-08
04-08
09-08
13-08
15-08
08-11
27-11
28-11
(5167, 15)


,RN,CLIENTE,CD_ALOCACAO,DATA_HORA_VAZAO,TURNO,DATA_SAIDA_CAMINHAO,DATA_CHEGADA_CAMINHAO,DESCRICAO_TIPO_CARGA,PESO_PADRAO,DISTANCIA,VAZAO_FRENTE,CLASSIFICACAO,TEMPO_INFERIOR,MARGEM_INFERIOR,MARGEM_SUPERIOR
1,1,CERRADINHO,574698,2024-03-25 18:52:00.000,MANHA,2024-03-25 18:53:04.000,2024-03-25 22:08:01.000000,RODOTREM PROPRIO,60,63.41,119.080,D,57.016667,7.016250,16.371250
2,1,CERRADINHO,574699,2024-03-25 18:56:00.000,MANHA,2024-03-25 18:57:00.000,2024-03-25 22:17:00.000000,RODOTREM PROPRIO,60,63.41,89.310,A,8.983333,7.016250,16.371250
3,1,CERRADINHO,574700,2024-03-25 18:58:00.000,MANHA,2024-03-25 18:58:50.000,2024-03-25 22:17:48.000000,RODOTREM PROPRIO,60,63.41,52.200,E,0.800000,7.016250,16.371250
4,1,CERRADINHO,574701,2024-03-25 19:24:00.000,NOITE,2024-03-25 19:24:32.000,2024-03-25 22:38:49.000000,RODOTREM PROPRIO,60,63.41,164.345,C,21.016667,7.016250,16.371250
5,1,CERRADINHO,574702,2024-03-25 19:38:00.000,NOITE,2024-03-25 19:38:28.000,2024-03-25 22:40:00.000000,RODOTREM PROPRIO,60,63.41,164.345,E,1.183333,7.016250,16.371250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5184,1,CERRADINHO,641804,2024-11-28 17:53:00.000,MANHA,2024-11-28 17:53:29.000,2024-11-28 22:15:16.000000,RODOTREM TERCEIRO,60,70.58,538.820,B,4.450000,5.489915,12.809802
5185,1,CERRADINHO,641802,2024-11-28 17:31:00.000,MANHA,2024-11-28 17:31:47.000,2024-11-28 22:20:11.000000,RODOTREM PROPRIO,60,70.58,555.670,B,4.916667,5.489915,12.809802
5186,1,CERRADINHO,641799,2024-11-28 17:11:00.000,MANHA,2024-11-28 17:12:24.000,2024-11-28 22:29:44.000000,RODOTREM PROPRIO,60,70.58,550.210,A,9.550000,5.489915,12.809802
5187,1,CERRADINHO,641805,2024-11-28 17:53:00.000,MANHA,2024-11-28 17:53:54.000,2024-11-28 23:17:02.000000,RODOTREM PROPRIO,60,70.58,538.820,D,47.300000,5.489915,12.809802
